In [95]:
import nltk
from collections import Counter
from nltk import ngrams
from Levenshtein import distance
import string
from nltk import pos_tag
from gensim.models import KeyedVectors
from collections import defaultdict

# Download necessary NLTK data
nltk.download('words')
nltk.download('brown')

from nltk.corpus import words, brown

[nltk_data] Downloading package words to /home/nathalie/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package brown to /home/nathalie/nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [96]:
# Load vocabulary and word frequencies
vocabulary = set(words.words())  # List of valid words
word_frequencies = Counter(brown.words())  # Word frequencies from the Brown corpus

In [97]:
brown.words()

['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', ...]

In [98]:
len(brown.words())

1161192

In [99]:
def preprocess_text(text):
    """Remove punctuation and lowercase text."""
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator).lower()

In [100]:
input = 'I have a cta'

input_preprocessed = preprocess_text(input)

In [101]:
input_preprocessed

'i have a cta'

In [102]:
tokens = [word.lower() for word in brown.words()]

In [103]:
len(tokens)

1161192

In [104]:
model = Counter(ngrams(tokens, 3))

In [105]:
model

Counter({("''", '?', '?'): 915,
         ("''", '.', '``'): 702,
         (',', 'and', 'the'): 656,
         ('.', 'it', 'is'): 529,
         ('.', 'it', 'was'): 518,
         ('one', 'of', 'the'): 403,
         ('.', 'in', 'the'): 401,
         ('?', '?', '``'): 381,
         ('.', '``', 'i'): 359,
         ("''", '!', '!'): 350,
         ("''", ',', 'he'): 346,
         ('.', 'he', 'was'): 341,
         (';', ';', 'and'): 341,
         ('the', 'united', 'states'): 336,
         (',', 'however', ','): 321,
         ("''", '.', 'the'): 318,
         ('said', '.', '``'): 276,
         (';', ';', 'the'): 273,
         (',', 'in', 'the'): 269,
         ('said', ',', '``'): 268,
         (',', 'he', 'said'): 257,
         ('as', 'well', 'as'): 238,
         ("''", '.', 'he'): 234,
         ('.', 'there', 'was'): 228,
         (',', 'it', 'is'): 228,
         (',', 'and', 'he'): 225,
         ('.', 'he', 'had'): 223,
         ('.', 'this', 'is'): 218,
         ('of', 'course', ','): 218,
  

In [106]:
len(model)

895057

In [107]:
# Generate suggestions for misspelled words
def generate_suggestions(word, vocabulary, word_frequencies, max_candidates=5):
    """Generate word suggestions based on edit distance and frequency."""
    suggestions = [
        (vocab_word, distance(word, vocab_word), word_frequencies[vocab_word])
        for vocab_word in vocabulary
    ]
    # Sort by edit distance (asc), then by frequency (desc)
    suggestions = sorted(suggestions, key=lambda x: (x[1], -x[2]))
    return [suggestion[0] for suggestion in suggestions[:max_candidates]]

In [108]:
word = 'hoese'
suggestions = [
        (vocab_word, distance(word, vocab_word), word_frequencies[vocab_word])
        for vocab_word in vocabulary
    ]

In [109]:
suggestions = sorted(suggestions, key=lambda x: (x[1], -x[2]))

In [110]:
suggestions[:10]

[('house', 1, 400),
 ('horse', 1, 110),
 ('hose', 1, 8),
 ('hoose', 1, 0),
 ('hoise', 1, 0),
 ('these', 2, 1228),
 ('those', 2, 782),
 ('here', 2, 607),
 ('home', 2, 526),
 ('does', 2, 445)]

In [111]:
suggestions = [suggestion[0] for suggestion in suggestions[0:5]]

In [112]:
suggestions

['house', 'horse', 'hose', 'hoose', 'hoise']

In [113]:
input = 'cheking speling is complicatek'

In [114]:
input_preprocessed = preprocess_text(input)
input_preprocessed

'cheking speling is complicatek'

In [115]:
words = input_preprocessed.split()

In [116]:
words

['cheking', 'speling', 'is', 'complicatek']

In [117]:
corrected_words = []
prev_word = None

for i, word in enumerate(words):
    if word in vocabulary:
        corrected_words.append(word)
        continue
    else:
        suggestions = [
        (vocab_word, distance(word, vocab_word), word_frequencies[vocab_word])
        for vocab_word in vocabulary]
        suggestions = sorted(suggestions, key=lambda x: (x[1], -x[2]))
        suggestions = [suggestion[0] for suggestion in suggestions[0:5]]
    if prev_word:
        best_suggestion = suggestions[0]
        best_prob = 0  # Initialize best probability

        for suggestion in suggestions:
            bigram_count = model.get((prev_word, suggestion), 0)
            unigram_count = sum(1 for bigram in model if bigram[0] == prev_word)
            
            # Calculate the N-gram probability
            prob = bigram_count / unigram_count if unigram_count > 0 else 0

            # Update best suggestion based on probability
            if prob > best_prob:
                best_prob = prob
                best_suggestion = suggestion
    else:
        best_suggestion = suggestions[0]
    corrected_words.append(best_suggestion) 
    # Update prev_word to the current word for the next iteration
    prev_word = word




In [118]:
" ".join(corrected_words)

'choking spelling is complicated'

In [119]:

# Load vocabulary and word frequencies
# vocabulary = set(words.words())  # List of valid words
word_frequencies = Counter(brown.words())  # Word frequencies from the Brown corpus

model = Counter(ngrams(tokens, 3))

# input = 'cheking speling is complicatek'
# input = 'my mathwr is angru'
# input = 'the bes are vry fastt whn they are flyinh'
# input = 'the girl has multipe dresses'
# input = 'korrectud'
input = 'i am beginning to wamder if lonjer sentinces wuld work bether for tis prgramn'
input_preprocessed = preprocess_text(input)
words = input_preprocessed.split()

corrected_words = []
prev_word = None

for i, word in enumerate(words):
    # if word in vocabulary:
    #     corrected_words.append(word)
    #     prev_word = word
    #     continue

    word_tag = pos_tag([word])[0][1]
    print(word, word_tag)
    # Generate suggestions
    suggestions = [
        (vocab_word, distance(word, vocab_word), word_frequencies[vocab_word])
        for vocab_word in vocabulary
    ]
    suggestions = sorted(suggestions, key=lambda x: (x[1], -x[2]))
    suggestions = [suggestion[0] for suggestion in suggestions[0:5]]
    print(suggestions)

    # Initialize variables
    best_suggestion = suggestions[0]
    best_combined_prob = 0

    # Get the next word (if it exists)
    next_word = words[i + 1] if i + 1 < len(words) else None

    for suggestion in suggestions:
        # Calculate bigram probability with the previous word
        prob_prev = 0
        if prev_word:
            bigram_count = model.get((prev_word, suggestion), 0)
            unigram_count = sum(1 for bigram in model if bigram[0] == prev_word)
            prob_prev = bigram_count / unigram_count if unigram_count > 0 else 0

        # Calculate bigram probability with the next word
        prob_next = 0
        if next_word:
            bigram_count = model.get((suggestion, next_word), 0)
            unigram_count = sum(1 for bigram in model if bigram[0] == suggestion)
            prob_next = bigram_count / unigram_count if unigram_count > 0 else 0

        # Combine probabilities (you can adjust weights as needed)
        combined_prob = (prob_prev + prob_next) / 2

        # Update best suggestion based on combined probability
        if combined_prob > best_combined_prob:
            best_combined_prob = combined_prob
            best_suggestion = suggestion

    corrected_words.append(best_suggestion)
    prev_word = best_suggestion  # Update previous word

corrected_text = ' '.join(corrected_words)
print(corrected_text)

i NN
['i', 'a', 'in', 'is', 'it']
am VBP
['am', 'a', 'as', 'at', 'an']
beginning VBG
['beginning', 'ginning', 'unbeginning', 'binning', 'rebeginning']
to TO
['to', 'no', 'so', 'two', 'do']
wamder NN
['wander', 'warder', 'wadder', 'wader', 'water']
if IN
['if', 'of', 'in', 'is', 'it']
lonjer NN
['longer', 'lower', 'wonder', 'lover', 'locker']
sentinces NNS
['sentence', 'sentinel', 'sentencer', 'Bentincks', 'sentience']
wuld NN
['would', 'wild', 'wold', 'weld', 'auld']
work NN
['work', 'word', 'works', 'wore', 'worn']
bether NN
['better', 'bother', 'ether', 'nether', 'becher']
for IN
['for', 'or', 'For', 'far', 'form']
tis NN
['is', 'his', 'this', 'tip', 'tie']
prgramn NN
['program', 'predamn', 'grain', 'organ', 'gram']
i am beginning to wander if longer sentence would work better for is program


In [120]:
from gensim.models import KeyedVectors
import numpy as np
from scipy.spatial.distance import cosine

word2vec = KeyedVectors.load_word2vec_format('/home/nathalie/Desktop/projects/spellchecker/models/GoogleNews-vectors-negative300.bin.gz', binary=True)


In [121]:

# Sentence and candidates
sentence = "my *blank* is angry"
candidates = ["rather", "mother", "matter"]

# Define the context words (excluding the blank)
context_words = ["my", "is", "waiting", "for", "me", "at", "home"]

# Get embeddings for context words
context_vectors = [word2vec[word] for word in context_words if word in word2vec]

# Compute the average context vector
context_vector = np.mean(context_vectors, axis=0)

# Compute similarity with candidate words
similarities = {}
for candidate in candidates:
    if candidate in word2vec:
        candidate_vector = word2vec[candidate]
        similarity = 1 - cosine(context_vector, candidate_vector)  # Cosine similarity
        similarities[candidate] = similarity

# Find the best-fitting word
best_fit = max(similarities, key=similarities.get)
print(f"The best-fitting word is: {best_fit}")


The best-fitting word is: mother


In [122]:
from collections import Counter
from nltk.util import ngrams
from nltk.corpus import brown
from nltk.metrics.distance import edit_distance
from scipy.spatial.distance import cosine
import numpy as np

# Example preprocess function
# def preprocess_text(text):
#     text = text.lower()
#     text = ''.join([char if char.isalnum() else ' ' for char in text])
#     return text

# # Initialize variables
# vocabulary = set(brown.words())  # Vocabulary from the Brown corpus
# word_frequencies = Counter(brown.words())  # Word frequencies from the Brown corpus
# tokens = preprocess_text(' '.join(brown.words())).split()
# model = Counter(ngrams(tokens, 3))  # Trigrams model



input = 'cheking speling is complicatek'
# input = 'my mathwr is angru'
# input = 'the bes are vry fastt whn they are flyinh'
# input = 'the girl has multipe dresses'
# input = 'i am beginning to wamder if lonjer sentinces wuld work bether for tis prgramn'
input_preprocessed = preprocess_text(input)
words = input_preprocessed.split()

corrected_words = []
prev_word = None

for i, word in enumerate(words):
    if word in vocabulary:
        corrected_words.append(word)
        prev_word = word
        continue
    print(f'current word: {word}')
    # word_tag = pos_tag([word])[0][1]
    # print(word, word_tag)
    # Generate suggestions
    suggestions = [
        (vocab_word, distance(word, vocab_word), word_frequencies[vocab_word])
        for vocab_word in vocabulary
    ]
    suggestions = sorted(suggestions, key=lambda x: (x[1], -x[2]))
    suggestions = [suggestion[0] for suggestion in suggestions[0:5]]
    print(suggestions)

    # Initialize variables
    best_suggestion = suggestions[0]
    best_combined_prob = 0

    # Get the next word (if it exists)
    next_word = words[i + 1] if i + 1 < len(words) else None

    for suggestion in suggestions:
        # Calculate bigram probability with the previous word
        prob_prev = 0
        if prev_word:
            bigram_count = model.get((prev_word, suggestion), 0)
            unigram_count = sum(1 for bigram in model if bigram[0] == prev_word)
            prob_prev = bigram_count / unigram_count if unigram_count > 0 else 0

        # Calculate bigram probability with the next word
        prob_next = 0
        if next_word:
            bigram_count = model.get((suggestion, next_word), 0)
            unigram_count = sum(1 for bigram in model if bigram[0] == suggestion)
            prob_next = bigram_count / unigram_count if unigram_count > 0 else 0

        # Combine probabilities (you can adjust weights as needed)
        combined_prob = (prob_prev + prob_next) / 2

        # Update best suggestion based on combined probability
        if combined_prob > best_combined_prob:
            best_combined_prob = combined_prob
            best_suggestion = suggestion

    corrected_words.append(best_suggestion)
    prev_word = best_suggestion  # Update previous word

corrected_text = ' '.join(corrected_words)
print(corrected_text, '\n\n')


context_words = corrected_text.split()
# context_vectors = [word2vec[word] for word in context_words if word in word2vec]
# context_vector = np.mean(context_vectors, axis=0)


for word in context_words:
    print(f'current word: {word}')
    print(f'context words: {context_words}')

    candidates = [
        (vocab_word, edit_distance(word, vocab_word), word_frequencies[vocab_word])
        for vocab_word in vocabulary
    ]
    candidates = sorted(candidates, key=lambda x: (x[1], -x[2]))
    candidates = [suggestion[0] for suggestion in candidates[:5]]
    print(candidates)

    # if word in context_words:
    context_words2 = context_words.copy()
    context_words2 = context_words2.remove(word)

    context_vectors = [word2vec[word] for word in context_words2 if word in word2vec]
    context_vector = np.mean(context_vectors, axis=0)
    similarities = {}
    for candidate in candidates:
        if candidate in word2vec:
            candidate_vector = word2vec[candidate]
            similarity = 1 - cosine(context_vector, candidate_vector)
            similarities[candidate] = similarity

    best_fit = max(similarities, key=similarities.get)
    print(f"The best-fitting word is: {best_fit}\n")
    if word != best_fit:
    # Replace the word in the context_words list (which is a list of words, not word vectors)
        index = context_words.index(word)
        context_words[index] = best_fit

corrected_text = ' '.join(context_words)
print(corrected_text, '\n\n')


current word: cheking
['choking', 'seeking', 'cooking', 'shaking', 'chasing']
current word: speling
['spelling', 'spewing', 'spiling', 'apeling', 'spelding']
current word: complicatek
['complicated', 'complicate', 'coimplicate', 'complicity', 'complication']
choking spelling is complicated 


current word: choking
context words: ['choking', 'spelling', 'is', 'complicated']
['choking', 'cooking', 'coking', 'coming', 'looking']


TypeError: 'NoneType' object is not iterable

In [ ]:
corrected_text = 'my mathwr is angru'

context_words = corrected_text.split()
context_vectors = [word2vec[word] for word in context_words if word in word2vec]
context_vector = np.mean(context_vectors, axis=0)


for word in context_words:
    print(f'current word: {word}')
    print(f'context words: {context_words}')
    candidates = [
        (vocab_word, edit_distance(word, vocab_word), word_frequencies[vocab_word])
        for vocab_word in vocabulary
    ]
    candidates = sorted(candidates, key=lambda x: (x[1], -x[2]))
    candidates = [suggestion[0] for suggestion in candidates[:5]]
    print(candidates)
    similarities = {}
    for candidate in candidates:
        if candidate in word2vec:
            candidate_vector = word2vec[candidate]
            similarity = 1 - cosine(context_vector, candidate_vector)
            similarities[candidate] = similarity

    best_fit = max(similarities, key=similarities.get)
    print(f"The best-fitting word is: {best_fit}\n")
    if word != best_fit:
    # Replace the word in the context_words list (which is a list of words, not word vectors)
        index = context_words.index(word)
        context_words[index] = best_fit

corrected_text = ' '.join(context_words)
print(corrected_text, '\n\n')


current word: my
context words: ['my', 'mathwr', 'is', 'angru']
['my', 'by', 'may', 'me', 'By']
The best-fitting word is: my

current word: mathwr
context words: ['my', 'mathwr', 'is', 'angru']
['rather', 'matter', 'mother', 'father', 'Father']
The best-fitting word is: matter

current word: is
context words: ['my', 'matter', 'is', 'angru']
['is', 'in', 'it', 'as', 'his']
The best-fitting word is: is

current word: angru
context words: ['my', 'matter', 'is', 'angru']
['angry', 'angle', 'anger', 'angel', 'Angry']
The best-fitting word is: angle

my matter is angle 




In [ ]:

# Sentence and candidates
sentence = "my *blank* is angry"
candidates = ['rather', 'matter', 'mother', 'father', 'Father']

# Define the context words (excluding the blank)
context_words = ["my", "is", "waiting", "for", "me", "at", "home"]

# Get embeddings for context words
context_vectors = [word2vec[word] for word in context_words if word in word2vec]

# Compute the average context vector
context_vector = np.mean(context_vectors, axis=0)

# Compute similarity with candidate words
similarities = {}
for candidate in candidates:
    if candidate in word2vec:
        candidate_vector = word2vec[candidate]
        similarity = 1 - cosine(context_vector, candidate_vector)  # Cosine similarity
        similarities[candidate] = similarity

# Find the best-fitting word
best_fit = max(similarities, key=similarities.get)
print(f"The best-fitting word is: {best_fit}")

The best-fitting word is: mother
